In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


# Importing the libraries 

In [3]:
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf

# Loading the dataset

In [5]:
df= pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [6]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


#  Split the data into training, validation, and testing sets

In [10]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)


# Load the tokenizer and encode the data

In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_texts = train_data['review'].tolist()
train_labels = train_data['sentiment'].tolist()
train_labels = [1 if label == "positive" else 0 for label in train_labels]
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

val_texts = val_data['review'].tolist()
val_labels = val_data['sentiment'].tolist()
val_labels = [1 if label == "positive" else 0 for label in val_labels]
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

test_texts = test_data['review'].tolist()
test_labels = test_data['sentiment'].tolist()
test_labels = [1 if label == "positive" else 0 for label in test_labels]
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Convert the encoded inputs to TensorFlow datasets

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(10000).batch(16)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), val_labels)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(64)

# Load and fine-tune the model

In [14]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
2000/2000 [==============================] - 1158s 565ms/step - loss: 0.2481 - accuracy: 0.8990 - val_loss: 0.1937 - val_accuracy: 0.9218
Epoch 2/3
2000/2000 [==============================] - 1127s 563ms/step - loss: 0.1339 - accuracy: 0.9523 - val_loss: 0.2151 - val_accuracy: 0.9314
Epoch 3/3
2000/2000 [==============================] - 1127s 564ms/step - loss: 0.0729 - accuracy: 0.9754 - val_loss: 0.2147 - val_accuracy: 0.9240


#  Evaluate the model on the testing set

In [15]:
results = model.evaluate(test_dataset)
print("test loss, test accuracy:", results)

157/157 [==============================] - 100s 636ms/step - loss: 0.2129 - accuracy: 0.9261
test loss, test accuracy: [0.21293900907039642, 0.9261000156402588]
